<a href="https://colab.research.google.com/github/a-gasior/Robot-Self-Localization/blob/master/Robot_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Andrew Gasiorowski

In [0]:
import numpy as np
import itertools
from functools import reduce

In [0]:
class Node:
    """Definition of a node"""
    def __init__(self, nm, value):
        self.name = nm
        self.west = self.north = self.east = self.south = False
        #Node.direction returns false if wall
        self.probability = 0
        self.value = value
        self.node_of_interest = False
        #Node of interest may come in handy for printing
        #stores information about transition probabilities in the order W,N,E
        self.space_num = False
        #if node is an open space this represents which space it is from left to right top to bottom
        self.transition_north = False
        self.transition_east = False



    def node_type(self):
        if self.node_of_interest:
            return node_of_interest
        return self.value

    def get_truth_vector(self):
        '''returns binary vector of surroundings'''
        return np.array([1 if self.west == False else 0,
                    1 if self.north == False else 0,
                    1 if self.east == False else 0,
                    1 if self.south == False else 0])
      
    def calculate_north_transition_probabilities(self,graph):
        '''~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ NORTH TRANSITION MATRIX ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'''
        '''NORTH CLEAR      NO WALL              (graph[graph[n].west].space_num, 0.1)      (graph[graph[n].north].space_num, 0.8)  (graph[graph[n].east].space_num, 0.1)'''
        '''NORTH CLEAR      WALL EAST            (graph[graph[n].west].space_num, 0.1)      (graph[graph[n].north].space_num, 0.8)  (graph[n].space_num, 0.1)'''
        '''NORTH CLEAR      WALL WEST            (graph[n].space_num, 0.1)                  (graph[graph[n].north].space_num, 0.8)  (graph[graph[n].east].space_num, 0.1)'''
        '''NORTH CLEAR      WALL EAST & WEST     (graph[n].space_num, 0.2)                  (graph[graph[n].north].space_num, 0.8)   '''


        '''NORTH BLOCKED    NO WALL              (graph[graph[n].west].space_num, 0.1)      (graph[n].space_num, 0.8)  (graph[graph[n].east].space_num, 0.1)'''
        '''NORTH BLOCKED    WALL EAST            (graph[graph[n].west].space_num, 0.1)      (graph[n].space_num, 0.1+0.8)'''
        '''NORTH BLOCKED    WALL WEST            (graph[n].space_num, 0.1+0.8)              (graph[graph[n].east].space_num, 0.1)'''
        '''NORTH BLOCKED    WALL EAST & WEST     (graph[n].space_num, 1.0)                     '''
        if self.north:
            '''NORTH CLEAR'''
            if not self.east and not self.west:
                '''WALL: EAST & WEST'''
                self.transition_north = [(graph[self.name].space_num, 0.2), (graph[graph[self.name].north].space_num, 0.8)]
            elif not self.east and self.west:
                '''WALL: EAST'''
                self.transition_north = [(graph[graph[self.name].west].space_num, 0.1), (graph[graph[self.name].north].space_num, 0.8),(graph[self.name].space_num, 0.1)]
            elif not self.west and self.east:
                '''WALL: WEST'''
                self.transition_north = [(graph[self.name].space_num, 0.1),(graph[graph[self.name].north].space_num, 0.8),(graph[graph[self.name].east].space_num, 0.1)]
            else:
                '''NO EAST OR WEST WALL'''
                self.transition_north = [(graph[graph[self.name].west].space_num, 0.1),(graph[graph[self.name].north].space_num, 0.8),(graph[graph[self.name].east].space_num, 0.1)]
        else:
            '''WALL NORTH'''
            if not self.east and not self.west:
                '''WALL: EAST & WEST'''
                self.transition_north = [(graph[self.name].space_num, 1.0)]
            elif not self.east and self.west:
                '''WALL: EAST'''
                self.transition_north = [(graph[graph[self.name].west].space_num, 0.1),(graph[self.name].space_num, 0.1+0.8)]
            elif not self.west and self.east:
                '''WALL: WEST'''
                self.transition_north = [(graph[self.name].space_num, 0.1+0.8),(graph[graph[self.name].east].space_num, 0.1)]
            else:
                '''NO EAST OR WEST WALL'''
                self.transition_north = [(graph[graph[self.name].west].space_num, 0.1),(graph[self.name].space_num, 0.8),(graph[graph[self.name].east].space_num, 0.1)]

    def calculate_east_transition_probabilities(self,graph):
        '''~~~~~~~~~~~~~EAST TRANSITION MATRIX~~~~~~~~~~~~~'''
        '''EAST CLEAR       NO WALL                 (graph[graph[n].east].space_num, 0.8)   (graph[graph[n].south].space_num, 0.1)  (graph[graph[n].north].space_num, 0.1)'''
        '''EAST CLEAR       NORTH WALL              (graph[graph[n].east].space_num, 0.8)   (graph[graph[n].south].space_num, 0.1)  (graph[n].space_num, 0.1)'''
        '''EAST CLEAR       SOUTH WALL              (graph[graph[n].east].space_num, 0.8)   (graph[graph[n].north].space_num, 0.1)  (graph[n].space_num, 0.1)'''
        '''EAST CLEAR       NORTH & SOUTH WALL      (graph[graph[n].east].space_num, 0.8)   (graph[n].space_num, 0.2)'''
        
        '''EAST BLOCKED     NO WALL                 (graph[n].space_num, 0.8)   (graph[graph[n].south].space_num, 0.1)  (graph[graph[n].north].space_num, 0.1)'''
        '''EAST BLOCKED     NORTH WALL              (graph[n].space_num, 0.9)   (graph[graph[n].south].space_num, 0.1)'''
        '''EAST BLOCKED     SOUTH WALL              (graph[n].space_num, 0.9)   (graph[graph[n].north].space_num, 0.1)'''
        '''EAST BLOCKED     NORTH & SOUTH WALL      (graph[n].space_num, 1.0)'''
        if self.east:
            if self.north and self.south:
                self.transition_east = [(graph[graph[self.name].east].space_num, 0.8),(graph[graph[self.name].south].space_num, 0.1),(graph[graph[self.name].north].space_num, 0.1)]
            elif self.south and not self.north:
                self.transition_east = [(graph[graph[self.name].east].space_num, 0.8),(graph[graph[self.name].south].space_num, 0.1),(graph[self.name].space_num, 0.1)]
            elif self.north and not self.south:
                self.transition_east = [(graph[graph[self.name].east].space_num, 0.8),(graph[graph[self.name].north].space_num, 0.1),(graph[self.name].space_num, 0.1)]
            else:
                self.transition_east = [(graph[graph[self.name].east].space_num, 0.8),(graph[self.name].space_num, 0.2)]
        else:
            if self.north and self.south:
                self.transition_east = [(graph[self.name].space_num, 0.8),(graph[graph[self.name].south].space_num, 0.1),(graph[graph[self.name].north].space_num, 0.1)]
            elif self.south and not self.north:
                self.transition_east = [(graph[self.name].space_num, 0.9),(graph[graph[self.name].south].space_num, 0.1)]
            elif self.north and not self.south:
                self.transition_east = [(graph[self.name].space_num, 0.9),(graph[graph[self.name].north].space_num, 0.1)]
            else:
                self.transition_east = [(graph[self.name].space_num, 1.0)]

class Robot:
    """This is a robot that moves through the maze"""
    def __init__(self, starting_loc):
        self.loc = starting_loc
        self.img = ':)'
    def get_loc(self):
        return self.loc
    def get_img(self):
        return self.img
    def move(self, dir, graph):
        cur = self.loc
        self.loc = getattr(graph[cur],dir) if graph[getattr(graph[cur],dir)].value == '_' else self.loc
        #update location to the node in the direction of travel if this node is an open space else 'bounce back'

In [0]:
def define_graph_features():
    """visual representation of the graph we will build"""
    """any graph can be built. X represents wall. _ open space"""
    r0 = ['X','X','X','X','X','X','X','X']
    r1 = ['X','_','_','_','_','_','_','X']
    r2 = ['X','_','X','X','X','X','_','X']
    r3 = ['X','_','X','_','_','X','_','X']
    r4 = ['X','_','X','_','_','_','_','X']
    r5 = ['X','_','_','_','_','_','_','X']
    r6 = ['X','X','X','X','X','X','X','X']
    maze = [r0, r1, r2, r3, r4, r5, r6]
    return maze

def build_graph():
    """Uses graphical representation of graph to construct a dictionary of nodes"""
    """dictionary takes the nodeID(int) and returns the node(object)"""
    open_spaces_dict = {}
    graph_list = define_graph_features()
    num_rows = len(graph_list)
    num_cols = len(graph_list[0])
    current = 0
    graph_dict = {}
    space_num = 0
    for r_idx, row in enumerate(graph_list):
        for c_idx, column in enumerate(row):
            new_node = Node(current, column)
            if new_node.value == '_':
                new_node.nth_node = space_num
                open_spaces_dict[space_num] = new_node#graph_list[current]####
                new_node.space_num = space_num
                space_num+=1
            new_node.west = check_west(graph_list[r_idx], c_idx, r_idx, num_cols)
            new_node.east = check_east(graph_list[r_idx], c_idx, r_idx, num_cols)
            new_node.north = check_north(graph_list[r_idx-1], c_idx, r_idx-1, num_cols) if r_idx != 0 else False
            new_node.south = check_south(graph_list[r_idx+1], c_idx, r_idx+1, num_cols) if r_idx < num_rows - 1 else False
            graph_dict[current] = new_node
            current += 1
    return graph_dict, graph_list, space_num, open_spaces_dict

def print_graph(graph, graph_list, robot):
    num_rows = len(graph_list)
    num_cols = len(graph_list[0])
    robot_location = robot.get_loc()
    for r in range(0,num_rows):
        for c in range(0,num_cols):
            cur_spot = r * num_cols + c
            print(robot.get_img(), end='\t') if cur_spot == robot_location else print(graph[cur_spot].node_type(),end='\t')
        print('\n')

def check_west(row, cur_idx, row_idx, num_cols):
    """Returns node to west of current"""
    if cur_idx == 0:
        return False
    if row[cur_idx-1] == 'X':
        return False
    return (row_idx * num_cols) + cur_idx - 1

def check_east(row, cur_idx, row_idx, num_cols):
    """Returns node to east of current"""
    if cur_idx == len(row) - 1:
        return False
    if row[cur_idx+1] == 'X':
        return False
    return (row_idx * num_cols) + cur_idx + 1

def check_north(north_row, cur_idx, row_idx, num_cols):
    """Returns node to north of current"""
    if north_row[cur_idx] == 'X':
        return False
    return (row_idx * num_cols) + cur_idx

def check_south(south_row, cur_idx, row_idx, num_cols):
    """Returns node to the south of current"""
    if south_row[cur_idx] == 'X':
        return False
    return (row_idx * num_cols) + cur_idx

In [0]:
def compute_sensor_matrix(sense, spaces_dict):
    '''Takes a sensor reading as arg1  and  the states as arg2   and   returns the sensor matrix (nx1)'''
    # sensitivity_matrix = {0:{0:0.95, 1:0.1}, 1:{0:0.05, 1:0.9}}
    sensitivity_matrix = {0:{0:0.95, 1:0.05}, 1:{0:0.1, 1:0.9}}
    sensor_matrix = []
    for state,_ in enumerate(spaces_dict):
        o_state = [[sensitivity_matrix[key] for key in sense][idx][ke] for idx,ke in enumerate(spaces_dict[state].get_truth_vector())]
        o_state = reduce(lambda x, y: x*y, o_state)
        sensor_matrix.append(o_state)
    return np.asarray(sensor_matrix)

def create_transition_matrix(num_spaces,spaces_dict,direction):
    '''Takes graph and returns an ndarray transition matrix TRANSPOSED'''
    transition_matrix = np.zeros((num_spaces,num_spaces))
    if direction == 'north':
        for from_idx in range(0,len(spaces_dict)):
            for to_idx in spaces_dict[from_idx].transition_north:
                transition_matrix[from_idx][to_idx[0]] = to_idx[1]
        return np.transpose(transition_matrix)
    elif direction =='east':
        for from_idx in range(0,len(spaces_dict)):
            for to_idx in spaces_dict[from_idx].transition_east:
                transition_matrix[from_idx][to_idx[0]] = to_idx[1]
        return np.transpose(transition_matrix)

# def calculate_normalization_matrix(sensor_matrix,transition_matrix,num_spaces):
#     normalization_matrix = []
#     for i in range(0,num_spaces):
#         normalization_matrix.append(1/np.dot(np.transpose(sensor_matrix),transition_matrix[i]))
#     # return np.diag(normalization_matrix)
#     values = np.diag(normalization_matrix)
#     nrm = (values - values.min()) / (values - values.min()).sum()
#     return nrm

def calculate_posterior(transition_matrix,sensor_matrix,prior):
    p1 = np.matmul(transition_matrix, prior)
    entire = np.matmul(np.diag(sensor_matrix),p1)
    return (1/np.sum(entire))*entire

In [0]:
robert = Robot(10)
graph, graph_list, num_spaces, spaces_dict = build_graph()
prior = np.transpose(np.full((1,23),1/num_spaces))
### steps = np.array(([0,0,0,0]])
steps = np.array([(np.array([0,0,0,0]),'north'),(np.array([1,0,0,0]),'north'),(np.array([1,1,0,0]),'north'),(np.array([0,1,1,0]),'east')])

for sense, mvmnt in steps:
    # sense = np.array([0,0,0,0])#do this sense
    sensor_matrix = compute_sensor_matrix(sense, spaces_dict)
    if mvmnt == 'north':
        for idx in range(0,len(spaces_dict)):
            spaces_dict[idx].calculate_north_transition_probabilities(graph)
    elif mvmnt == 'east':
        for idx in range(0,len(spaces_dict)):
            spaces_dict[idx].calculate_east_transition_probabilities(graph)
    transition_matrix = create_transition_matrix(num_spaces,spaces_dict,mvmnt)
    prior = calculate_posterior(transition_matrix,sensor_matrix,prior)
    for idx in range(0, len(spaces_dict)):
        spaces_dict[idx].node_of_interest = prior[idx]
    
    for rdx,row in enumerate(graph_list):
        for cdx,col in enumerate(row):
            graph_idx = len(row)*rdx+cdx
            if graph[graph_idx].node_of_interest:
                print('{:.1e}'.format(float(graph[graph_idx].node_of_interest)),end='\t\t')
                # print(graph[graph_idx].node_of_interest,end='\t')
            else:
                print(graph[graph_idx].value,end='\t\t')
        print('\n')

    print('\n\n')



X		X		X		X		X		X		X		X		

X		3.9e-03		2.2e-03		2.2e-03		2.2e-03		2.2e-03		3.9e-03		X		

X		2.2e-03		X		X		X		X		2.2e-03		X		

X		2.2e-03		X		3.9e-03		3.9e-03		X		2.2e-03		X		

X		2.2e-03		X		4.1e-02		7.8e-01		7.4e-02		4.1e-02		X		

X		4.3e-04		2.2e-03		8.2e-03		8.2e-03		8.2e-03		4.3e-04		X		

X		X		X		X		X		X		X		X		




X		X		X		X		X		X		X		X		

X		3.0e-03		7.5e-06		6.9e-06		6.9e-06		7.5e-06		1.7e-05		X		

X		1.2e-03		X		X		X		X		1.2e-03		X		

X		1.2e-03		X		2.0e-02		2.0e-03		X		1.8e-02		X		

X		4.3e-04		X		9.2e-01		2.1e-02		9.0e-03		7.2e-04		X		

X		1.4e-04		8.3e-06		6.3e-05		1.0e-04		5.3e-05		2.8e-06		X		

X		X		X		X		X		X		X		X		




X		X		X		X		X		X		X		X		

X		4.7e-03		2.3e-06		5.3e-08		5.3e-08		6.4e-08		7.3e-06		X		

X		9.0e-06		X		X		X		X		1.1e-04		X		

X		4.4e-06		X		9.8e-01		1.6e-04		X		3.2e-05		X		

X		1.5e-06		X		1.4e-02		1.5e-03		1.4e-03		8.2e-07		X		

X		1.1e-07		2.1e-07		9.1e-09		9.7e-09		8.6e-09		2.4e-10		X		

X		X		X		X		X		X		X		X		




X		X		X		X		X		X		X		X		

X		3.

In [0]:
for rdx,row in enumerate(graph_list):
    for cdx,col in enumerate(row):
        graph_idx = len(row)*rdx+cdx
        if graph[graph_idx].node_of_interest:
            print('{:.1e}'.format(float(graph[graph_idx].node_of_interest)),end='\t\t')
            # print(graph[graph_idx].node_of_interest,end='\t')
        else:
            print(graph[graph_idx].value,end='\t\t')
    print('\n')

    